In [1]:
import cv2
import numpy as np
import glob

In [2]:


# Definir el número de esquinas internas del tablero de ajedrez
chessboard_size = (8, 5)

# Preparar puntos de objeto como (0,0,0), (1,0,0), (2,0,0), ....,(8,5,0)
objp = np.zeros((chessboard_size[0] * chessboard_size[1], 3), np.float32)
objp[:, :2] = np.mgrid[0:chessboard_size[0], 0:chessboard_size[1]].T.reshape(-1, 2)

# Arrays para almacenar los puntos del objeto 3D y los puntos de la imagen 2D de todas las imágenes
objpoints = []  # puntos 3D en el mundo real
imgpoints = []  # puntos 2D en el plano de la imagen

# Cargar todas las imágenes de calibración
images = glob.glob('imagenes/*.jpg')

for fname in images:
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Encontrar las esquinas del tablero de ajedrez
    ret, corners = cv2.findChessboardCorners(gray, chessboard_size, None)

    # Si se encuentran, añadir puntos de objeto y puntos de imagen
    if ret:
        objpoints.append(objp)
        imgpoints.append(corners)

        # Dibujar y mostrar las esquinas del tablero de ajedrez
        cv2.drawChessboardCorners(img, chessboard_size, corners, ret)
        cv2.imshow('img', img)
        cv2.waitKey(100)

cv2.destroyAllWindows()

# Calibrar la cámara
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)

# Guardar los parámetros de calibración
np.savez('calibracion_camara.npz', ret=ret, mtx=mtx, dist=dist, rvecs=rvecs, tvecs=tvecs)

print("Calibración de la cámara completada.")


Calibración de la cámara completada.


In [3]:
# Cargar la calibración
calibration_data = np.load('calibracion_camara.npz')
mtx = calibration_data['mtx']
dist = calibration_data['dist']

# Cargar una imagen para de-distorsionar
img = cv2.imread('aruco2.jpg')
h, w = img.shape[:2]

# Obtener la nueva matriz de cámara y la región de interés
newcameramtx, roi = cv2.getOptimalNewCameraMatrix(mtx, dist, (w, h), 1, (w, h))

# De-distorsionar la imagen
dst = cv2.undistort(img, mtx, dist, None, newcameramtx)

# Recortar la imagen utilizando la región de interés
x, y, w, h = roi
dst = dst[y:y+h, x:x+w]

cv2.imshow('Imagen De-distorsionada2', dst)
cv2.imwrite('imagen_dedistorsionada2.jpg', dst)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [4]:
import cv2
import numpy as np
import cv2.aruco as aruco
import random

def create_aruco_marker(dict_type, marker_id, size):
    aruco_dict = aruco.getPredefinedDictionary(dict_type)
    img_marker = np.zeros((size, size), dtype=np.uint8)
    img_marker = aruco.drawMarker(aruco_dict, marker_id, size)
    return img_marker

# Parámetros para el marcador
dict_type = aruco.DICT_6X6_250
size = 700

# Generar un marker_id aleatorio entre 0 y el número máximo de IDs permitidos por el diccionario
max_ids = aruco.getPredefinedDictionary(dict_type).bytesList.shape[0] - 1
marker_id = random.randint(0, max_ids)

# Crear y guardar el marcador ARUCO
img_marker = create_aruco_marker(dict_type, marker_id, size)
cv2.imwrite(f'marcador_aruco_{marker_id}.png', img_marker)

# Mostrar el marcador
cv2.imshow(f'Marcador ARUCO ID {marker_id}', img_marker)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [21]:
"""import numpy as np
import cv2
import tensorflow as tf
import tensorflow_hub as hub
import winsound
import time

# Ajustar el tamaño del marcador en metros
tamaño_marcador = 0.17  # Tamaño del marcador ARUCO en metros

def reproducir_alarma():
    frecuencia = 1000  # Frecuencia en Hertz
    duracion = 1000    # Duración en milisegundos
    winsound.Beep(frecuencia, duracion)

def detectar_marcador_aruco_y_objetos(calibration_data_path, ip_camera_url):
    # Cargar los parámetros de la cámara desde la calibración previa
    calibration_data = np.load(calibration_data_path)
    mtx = calibration_data['mtx']
    dist = calibration_data['dist']

    # Inicializar la captura de video desde la cámara IP
    cap = cv2.VideoCapture(ip_camera_url)

    if not cap.isOpened():
        print("No se pudo abrir la cámara.")
        return

    # Cargar el modelo de detección de objetos desde TensorFlow Hub
    detector = hub.load("https://tfhub.dev/tensorflow/ssd_mobilenet_v2/2")

    alarma_activada = False
    tiempo_alarma = 0
    intervalo_alarma = 5  # Intervalo de tiempo en segundos entre alarmas

    while True:
        # Capturar un fotograma de video
        ret, frame = cap.read()
        if not ret:
            print("No se pudo leer el fotograma.")
            break

        # Convertir la imagen a escala de grises
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Detectar los marcadores ARUCO
        aruco_dict = cv2.aruco.getPredefinedDictionary(cv2.aruco.DICT_6X6_250)
        parameters = cv2.aruco.DetectorParameters_create()
        corners, ids, _ = cv2.aruco.detectMarkers(gray, aruco_dict, parameters=parameters)

        if ids is not None and len(ids) == 4:
            # Dibujar los bordes de los marcadores detectados
            frame = cv2.aruco.drawDetectedMarkers(frame, corners, ids)

            # Ordenar los marcadores por ID para mantener un orden consistente
            ids = ids.flatten()
            orden = np.argsort(ids)
            corners = [corners[i] for i in orden]

            # Obtener las posiciones de las esquinas de los marcadores
            puntos_rectangulo = np.array([corner[0][0] for corner in corners], dtype=np.int32)

            # Dibujar el rectángulo prohibido en la imagen
            cv2.polylines(frame, [puntos_rectangulo], isClosed=True, color=(0, 0, 255), thickness=2)

            # Detección de personas y objetos
            input_tensor = tf.convert_to_tensor(frame)
            input_tensor = input_tensor[tf.newaxis, ...]
            detections = detector(input_tensor)

            # Procesar las detecciones y verificar si están dentro de la región prohibida
            objeto_detectado = False
            for i in range(int(detections['num_detections'])):
                class_id = int(detections['detection_classes'][0][i])
                score = detections['detection_scores'][0][i]
                bbox = detections['detection_boxes'][0][i]
                if score > 0.5:  # Filtrar detecciones con confianza mayor a 50%
                    # Convertir de coordenadas relativas a píxeles
                    ymin, xmin, ymax, xmax = bbox
                    (startX, startY, endX, endY) = (int(xmin * frame.shape[1]), int(ymin * frame.shape[0]),
                                                    int(xmax * frame.shape[1]), int(ymax * frame.shape[0]))

                    # Dibujar la caja de detección en la imagen
                    cv2.rectangle(frame, (startX, startY), (endX, endY), (0, 255, 0), 2)

                    # Verificar si el objeto está dentro de la región prohibida
                    centro_objeto = ((startX + endX) // 2, (startY + endY) // 2)
                    if cv2.pointPolygonTest(puntos_rectangulo, centro_objeto, False) >= 0:
                        objeto_detectado = True
                        break

            # Manejar la alarma
            if objeto_detectado:
                if not alarma_activada or (time.time() - tiempo_alarma > intervalo_alarma):
                    reproducir_alarma()
                    alarma_activada = True
                    tiempo_alarma = time.time()
                print("¡Alarma! Objeto detectado en el área del rectángulo.")
            else:
                alarma_activada = False  # Resetear el estado de la alarma si no hay objeto detectado

        # Mostrar la imagen con el rectángulo prohibido proyectado y detecciones
        cv2.imshow('Detección de Objetos y Región Prohibida', frame)

        # Salir si se presiona la tecla 'q' o se cierra la ventana
        if cv2.getWindowProperty('Detección de Objetos y Región Prohibida', cv2.WND_PROP_VISIBLE) < 1:
            break

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # Liberar la captura y cerrar todas las ventanas
    cap.release()
    cv2.destroyAllWindows()


# Reemplaza 'http://192.168.1.100:8080/video' con la dirección IP de tu teléfono.
ip_camera_url = 'http://192.168.100.130:4747/video'
detectar_marcador_aruco_y_objetos('calibracion_camara.npz', ip_camera_url) 
"""


¡Alarma! Objeto detectado en el área del rectángulo.
¡Alarma! Objeto detectado en el área del rectángulo.
¡Alarma! Objeto detectado en el área del rectángulo.
¡Alarma! Objeto detectado en el área del rectángulo.
¡Alarma! Objeto detectado en el área del rectángulo.
¡Alarma! Objeto detectado en el área del rectángulo.
¡Alarma! Objeto detectado en el área del rectángulo.
¡Alarma! Objeto detectado en el área del rectángulo.
¡Alarma! Objeto detectado en el área del rectángulo.
¡Alarma! Objeto detectado en el área del rectángulo.
¡Alarma! Objeto detectado en el área del rectángulo.
No se pudo leer el fotograma.


In [29]:
import numpy as np
import cv2
import tensorflow as tf
import tensorflow_hub as hub
import winsound
import time

# Ajustar el tamaño del marcador en metros
tamaño_marcador = 0.17  # Tamaño del marcador ARUCO en metros

def reproducir_alarma():
    frecuencia = 1000  # Frecuencia en Hertz
    duracion = 1000    # Duración en milisegundos
    winsound.Beep(frecuencia, duracion)

def detectar_marcador_aruco_y_objetos(calibration_data_path, ip_camera_url):
    # Cargar los parámetros de la cámara desde la calibración previa
    calibration_data = np.load(calibration_data_path)
    mtx = calibration_data['mtx']
    dist = calibration_data['dist']

    # Inicializar la captura de video desde la cámara IP
    cap = cv2.VideoCapture(ip_camera_url)

    if not cap.isOpened():
        print("No se pudo abrir la cámara.")
        return

    # Cargar el modelo de detección de objetos desde TensorFlow Hub
    detector = hub.load("https://tfhub.dev/tensorflow/ssd_mobilenet_v2/2")

    alarma_activada = False
    tiempo_alarma = 0
    intervalo_alarma = 5  # Intervalo de tiempo en segundos entre alarmas

    while True:
        # Capturar un fotograma de video
        ret, frame = cap.read()
        if not ret:
            print("No se pudo leer el fotograma.")
            break

        # Convertir la imagen a escala de grises
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Detectar los marcadores ARUCO
        aruco_dict = cv2.aruco.getPredefinedDictionary(cv2.aruco.DICT_6X6_250)
        parameters = cv2.aruco.DetectorParameters_create()
        corners, ids, _ = cv2.aruco.detectMarkers(gray, aruco_dict, parameters=parameters)

        if ids is not None and len(ids) == 4:
            # Dibujar los bordes de los marcadores detectados
            frame = cv2.aruco.drawDetectedMarkers(frame, corners, ids)

            # Ordenar los marcadores por ID para mantener un orden consistente
            ids = ids.flatten()
            orden = np.argsort(ids)
            corners = [corners[i] for i in orden]

            # Obtener las posiciones de las esquinas de los marcadores
            puntos_rectangulo = np.array([corner[0][0] for corner in corners], dtype=np.int32)

            # Dibujar el rectángulo prohibido en la imagen
            cv2.polylines(frame, [puntos_rectangulo], isClosed=True, color=(0, 0, 255), thickness=2)

            # Detección de personas y objetos
            input_tensor = tf.convert_to_tensor(frame)
            input_tensor = input_tensor[tf.newaxis, ...]
            detections = detector(input_tensor)

            # Procesar las detecciones y verificar si están dentro de la región prohibida
            objeto_detectado = False
            persona_detectada = False
            for i in range(int(detections['num_detections'])):
                class_id = int(detections['detection_classes'][0][i])
                score = detections['detection_scores'][0][i]
                bbox = detections['detection_boxes'][0][i]
                if score > 0.5:  # Filtrar detecciones con confianza mayor a 50%
                    # Convertir de coordenadas relativas a píxeles
                    ymin, xmin, ymax, xmax = bbox
                    (startX, startY, endX, endY) = (int(xmin * frame.shape[1]), int(ymin * frame.shape[0]),
                                                    int(xmax * frame.shape[1]), int(ymax * frame.shape[0]))

                    # Dibujar la caja de detección en la imagen
                    cv2.rectangle(frame, (startX, startY), (endX, endY), (0, 255, 0), 2)

                    # Verificar si el objeto está dentro de la región prohibida
                    centro_objeto = ((startX + endX) // 2, (startY + endY) // 2)
                    if cv2.pointPolygonTest(puntos_rectangulo, centro_objeto, False) >= 0:
                        objeto_detectado = True
                        if class_id == 1:  # class_id 1 corresponde a 'person'
                            persona_detectada = True
                        break

            # Mostrar mensajes en la imagen
            if persona_detectada:
                cv2.putText(frame, "Persona detectada en la zona prohibida!", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1,
                            (0, 0, 255), 2, cv2.LINE_AA)
                if not alarma_activada or (time.time() - tiempo_alarma > intervalo_alarma):
                    reproducir_alarma()
                    alarma_activada = True
                    tiempo_alarma = time.time()
            elif objeto_detectado:
                cv2.putText(frame, "Objeto detectado, pero no es una persona", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1,
                            (255, 255, 0), 2, cv2.LINE_AA)
            else:
                alarma_activada = False  # Resetear el estado de la alarma si no hay objeto detectado

        # Mostrar la imagen con el rectángulo prohibido proyectado y detecciones
        cv2.imshow('Detección de Objetos y Región Prohibida', frame)

        # Salir si se presiona la tecla 'q' o se cierra la ventana
        if cv2.getWindowProperty('Detección de Objetos y Región Prohibida', cv2.WND_PROP_VISIBLE) < 1:
            break

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # Liberar la captura y cerrar todas las ventanasq
    cap.release()
    cv2.destroyAllWindows()


# Reemplaza 'http://192.168.1.100:8080/video' con la dirección IP de tu teléfono.
ip_camera_url = 'http://192.168.100.130:4747/video'
detectar_marcador_aruco_y_objetos('calibracion_camara.npz', ip_camera_url)


In [19]:
"""import numpy as np
import cv2
import tensorflow as tf
import tensorflow_hub as hub
import winsound
import time

# Ajustar el tamaño del marcador en metros
tamaño_marcador = 0.17  # Tamaño del marcador ARUCO en metros

def reproducir_alarma():
    frecuencia = 1000  # Frecuencia en Hertz
    duracion = 1000    # Duración en milisegundos
    winsound.Beep(frecuencia, duracion)

def detectar_marcador_aruco_y_objetos(calibration_data_path, ip_camera_url):
    # Cargar los parámetros de la cámara desde la calibración previa
    calibration_data = np.load(calibration_data_path)
    mtx = calibration_data['mtx']
    dist = calibration_data['dist']

    # Inicializar la captura de video desde la cámara IP
    cap = cv2.VideoCapture(ip_camera_url)

    if not cap.isOpened():
        print("No se pudo abrir la cámara.")
        return

    # Cargar el modelo de detección de objetos desde TensorFlow Hub
    detector = hub.load("https://tfhub.dev/tensorflow/ssd_mobilenet_v2/2")

    alarma_activada = False
    tiempo_alarma = 0
    intervalo_alarma = 5  # Intervalo de tiempo en segundos entre alarmas

    while True:
        # Capturar un fotograma de video
        ret, frame = cap.read()
        if not ret:
            print("No se pudo leer el fotograma.")
            break

        # Convertir la imagen a escala de grises
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Detectar los marcadores ARUCO
        aruco_dict = cv2.aruco.getPredefinedDictionary(cv2.aruco.DICT_6X6_250)
        parameters = cv2.aruco.DetectorParameters_create()
        corners, ids, _ = cv2.aruco.detectMarkers(gray, aruco_dict, parameters=parameters)

        if ids is not None and len(ids) == 4:
            # Dibujar los bordes de los marcadores detectados
            frame = cv2.aruco.drawDetectedMarkers(frame, corners, ids)

            # Ordenar los marcadores por ID para mantener un orden consistente
            ids = ids.flatten()
            orden = np.argsort(ids)
            corners = [corners[i] for i in orden]

            # Obtener las posiciones de las esquinas de los marcadores
            puntos_rectangulo = np.array([corner[0][0] for corner in corners], dtype=np.int32)

            # Dibujar el rectángulo prohibido en la imagen
            cv2.polylines(frame, [puntos_rectangulo], isClosed=True, color=(0, 0, 255), thickness=2)

            # Detección de personas y objetos
            input_tensor = tf.convert_to_tensor(frame)
            input_tensor = input_tensor[tf.newaxis, ...]
            detections = detector(input_tensor)

            # Procesar las detecciones y verificar si están dentro de la región prohibida
            objeto_detectado = False
            persona_detectada = False
            for i in range(int(detections['num_detections'])):
                class_id = int(detections['detection_classes'][0][i])
                score = detections['detection_scores'][0][i]
                bbox = detections['detection_boxes'][0][i]
                if score > 0.5:  # Filtrar detecciones con confianza mayor a 50%
                    # Convertir de coordenadas relativas a píxeles
                    ymin, xmin, ymax, xmax = bbox
                    (startX, startY, endX, endY) = (int(xmin * frame.shape[1]), int(ymin * frame.shape[0]),
                                                    int(xmax * frame.shape[1]), int(ymax * frame.shape[0]))

                    # Dibujar la caja de detección en la imagen
                    cv2.rectangle(frame, (startX, startY), (endX, endY), (0, 255, 0), 2)

                    # Verificar si el objeto está dentro de la región prohibida
                    centro_objeto = ((startX + endX) // 2, (startY + endY) // 2)
                    if cv2.pointPolygonTest(puntos_rectangulo, centro_objeto, False) >= 0:
                        objeto_detectado = True
                        if class_id == 1:  # class_id 1 corresponde a 'person'
                            persona_detectada = True
                        break

            # Mostrar mensajes en la imagen
            if persona_detectada:
                cv2.putText(frame, "Persona detectada en la zona prohibida!", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1,
                            (0, 0, 255), 2, cv2.LINE_AA)
                if not alarma_activada or (time.time() - tiempo_alarma > intervalo_alarma):
                    reproducir_alarma()
                    alarma_activada = True
                    tiempo_alarma = time.time()
            elif objeto_detectado:
                cv2.putText(frame, "Objeto detectado, pero no es una persona", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1,
                            (255, 255, 0), 2, cv2.LINE_AA)
                if not alarma_activada or (time.time() - tiempo_alarma > intervalo_alarma):
                    reproducir_alarma()
                    alarma_activada = True
                    tiempo_alarma = time.time()
            else:
                alarma_activada = False  # Resetear el estado de la alarma si no hay objeto detectado

        # Mostrar la imagen con el rectángulo prohibido proyectado y detecciones
        cv2.imshow('Detección de Objetos y Región Prohibida', frame)

        # Salir si se presiona la tecla 'q' o se cierra la ventana
        if cv2.getWindowProperty('Detección de Objetos y Región Prohibida', cv2.WND_PROP_VISIBLE) < 1:
            break

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # Liberar la captura y cerrar todas las ventanasq
    cap.release()
    cv2.destroyAllWindows()


# Reemplaza 'http://192.168.1.100:8080/video' con la dirección IP de tu teléfono.
ip_camera_url = 'http://192.168.100.130:4747/video'
detectar_marcador_aruco_y_objetos('calibracion_camara.npz', ip_camera_url)
"""
